# Perform face classification using VGGface

## 1. First resolving version conflict of the default vggface package

In [68]:
# Resolve version conflict from one of the files
filename = "/home/eric/anaconda3/envs/peekingduckling/lib/python3.8/site-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))

20951

In [72]:
%load_ext autoreload
%autoreload 2

In [76]:
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import numpy as np
from numpy import asarray
from PIL import Image
from tensorflow import keras
from scipy.spatial.distance import cosine

## 2. Download base model

In [34]:
# Loading VGGFace model without classifier and getting output to be a face embedding
base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
# summarize input and output shape
print('Inputs: %s' % base_model.inputs)
print('Outputs: %s' % base_model.outputs)

Inputs: [<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_6')>]
Outputs: [<KerasTensor: shape=(None, 2048) dtype=float32 (created by layer 'global_average_pooling2d_3')>]


## 3. Obtain image embeddings

In [35]:
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(files, model):
	# extract faces
	faces = [f for f in files]
	# convert into an array of samples
	samples = asarray(faces, 'float32')
	# prepare the face for the model, e.g. center pixels
	samples = preprocess_input(samples, version=2)
	# create a vggface model
	# perform prediction
	yhat = model.predict(samples)
	return yhat

In [36]:
model = keras.Sequential([
    keras.layers.Resizing(224,224),
    # keras.layers.RandomZoom((-0.2, 0.2), fill_mode='reflect'),
    # keras.layers.RandomTranslation(0.2, 0.2, fill_mode='reflect'),
    # keras.layers.RandomRotation(0.2, fill_mode='reflect'),
    # keras.layers.RandomFlip(),
    # keras.layers.RandomContrast(0.2),
    base_model,
])

## 4. Compare image embeddings between two different person

In [59]:
image_path = '../data/raw/images/clarence/300_clarence.jpg'
img = Image.open(image_path)

In [60]:
img = np.array(img)
clarence_embedding = get_embeddings([img], model)

In [61]:
clarence_embedding

array([[0.7209997 , 0.        , 0.03487726, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [62]:
image_path = '../data/raw/images/eric_kwok/0_eric_kwok.jpg'
img = Image.open(image_path)

In [63]:
img = np.array(img)
kwok_embedding = get_embeddings([img], model)

In [64]:
kwok_embedding

array([[2.646888 , 0.0573287, 0.       , ..., 5.8668456, 0.       ,
        0.       ]], dtype=float32)

In [65]:
# Comparison of embedding using euclidean distance
diff_dist = np.linalg.norm(clarence_embedding-kwok_embedding)
diff_dist

101.5161

In [66]:
# Comparring using cosine difference (1 - cosine distance)
cos_sim = cosine(clarence_embedding, kwok_embedding)
cos_sim

0.576074093580246

## 5. Compare embeddings for the same person

In [57]:
image_path = '../data/raw/images/clarence/300_clarence.jpg'
img = Image.open(image_path)

img = np.array(img)
clarence_embedding_2 = get_embeddings([img], model)

# Comparison of embedding
diff_dist = np.linalg.norm(clarence_embedding-clarence_embedding_2)
diff_dist

88.50315

In [58]:
# Comparring using cosine distance (1 - cosine similarity)
cos_sim = cosine(clarence_embedding, clarence_embedding_2)
cos_sim

0.46248912811279297

## 6. Get the embeddings from clarence, eric kwok and eric lee